# Convert Audio Format — Folder Batch

Converts all audio files in a folder to **mono 16-bit PCM WAV**.
Handles mp3, ogg, flac, m4a, aac, wma, amr, mp4, flv, and existing wav files.

---

> Uses `pydub` + system `ffmpeg`. No GPU required.

In [ ]:
import os

# ════════════════════════════════════════════
# ⚙️  CONFIGURATION
# ════════════════════════════════════════════
os.environ.setdefault("INPUT_DIR",   "/data/raw_audio")       # Source folder (any audio format)
os.environ.setdefault("OUTPUT_DIR",  "/data/converted_wav")   # Destination folder (WAV output)
os.environ.setdefault("TARGET_SR",   "")                      # Target sample rate (empty = keep original)
os.environ.setdefault("MONO",        "1")                     # 1 = force mono

INPUT_DIR  = os.environ["INPUT_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]
TARGET_SR  = os.environ["TARGET_SR"].strip()
MONO       = os.environ.get("MONO", "1").strip() in ("1", "true", "yes")

print(f"Input:     {INPUT_DIR}")
print(f"Output:    {OUTPUT_DIR}")
print(f"Target SR: {TARGET_SR or 'keep original'}")
print(f"Mono:      {MONO}")

In [ ]:
!pip install --quiet --break-system-packages pydub tqdm

In [ ]:
import os, shutil
from pathlib import Path
from pydub import AudioSegment
from tqdm import tqdm

SUPPORTED = {'.mp3', '.mp4', '.wma', '.ogg', '.flv', '.aac', '.m4a',
             '.amr', '.wav', '.flac', '.opus', '.webm'}

os.makedirs(OUTPUT_DIR, exist_ok=True)

files = [p for p in Path(INPUT_DIR).rglob("*") if p.suffix.lower() in SUPPORTED]
print(f"Found {len(files)} audio files")

converted, skipped, failed = 0, 0, 0
for src in tqdm(files, desc="Converting", unit="file"):
    rel = src.relative_to(INPUT_DIR)
    dst = Path(OUTPUT_DIR) / rel.with_suffix(".wav")
    dst.parent.mkdir(parents=True, exist_ok=True)

    if dst.exists():
        skipped += 1
        continue

    try:
        audio = AudioSegment.from_file(str(src))
        if MONO:
            audio = audio.set_channels(1)
        if TARGET_SR:
            audio = audio.set_frame_rate(int(TARGET_SR))
        audio = audio.set_sample_width(2)  # 16-bit
        audio.export(str(dst), format="wav")
        converted += 1
    except Exception as e:
        tqdm.write(f"  FAIL {src.name}: {e}")
        failed += 1

print(f"\nDone: {converted} converted, {skipped} skipped, {failed} failed")